### Backfill Features

In [ ]:
import os

import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 500)

import hopsworks

# change working directory to project root when running from notebooks folder to make it easier to import modules
# and to access sibling folders
os.chdir("..")

from src.data.cleaning import (
    process_games,
    add_TARGET,
)

from src.data.build_features import (
    process_features,
)

from src.utils.hopsworks_utils import (
    save_feature_names,
)

from src.utils.constants import (
    FEATURE_GROUP_VERSION,
)

from pathlib import Path  # for Windows/Linux compatibility

DATAPATH = Path(r"data")

In [ ]:
from dotenv import load_dotenv

load_dotenv()


try:
    HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
except:
    raise Exception("Set environment variable HOPSWORKS_API_KEY")

### Get Data

In [ ]:
games = pd.read_csv(DATAPATH / "interim" / "games.csv")

### Data Processing

In [ ]:
games = process_games(games)
games = add_TARGET(games)

### Feature Engineering

In [ ]:
# Feature engineering to add:
# rolling averages of key stats,
# win/lose streaks,
# home/away streaks,
# specific matchup (team X vs team Y) rolling averages and streaks

games = process_features(games)
games

### Backfill Feature Store

In [ ]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

**Rolling Stats Features**

In [ ]:
games

In [ ]:
rolling_stats_fg = fs.create_feature_group(
    name="rolling_stats",
    version=FEATURE_GROUP_VERSION,
    description="Rolling averages and current win/lose streaks",
    # primary_key=["GAME_ID"],
    primary_key=["GAME_DATE_EST", "HOME_TEAM_ID"],
    event_time="game_date_est",  # must be lowercase
)


rolling_stats_fg.insert(games, write_options={"wait_for_job": False})

**Save original feature names to JSON**

In [ ]:
# hopsworks "sanitizes" feature names by converting to all lowercase
# this function saves the original so that they can be re-mapped later
# for code re-usability

save_feature_names(games)